# Open AI Basics

## Lerninhalte

- Nutzung eines OpenAI API-Keys
- Kennen der OpenAI API
- Kennen der OpenAI Python-Bibliothek
- Senden eines dynamischen Prompts an die OpenAI API und Ausgabe der Antwort

## Anwendungsfall

- META: Kategorisierung und Auswahl von Elementen der richtigen Kategorie.
- Fall: Ein Kunde möchte eine Bestellung aufgeben, aber nicht direkt eine Pizza bestellen, sondern eine Zutat nennen. Die Aufgabe besteht darin, die passende Kategorie zu finden.

## Setup

Zunächst müssen wir die OpenAI Python-Bibliothek installieren. Dazu installieren wir gleich ein paar mehr Bibliotheken,
welche uns später das Programmieren leichter machen. Dies kann über den folgenden Befehl erfolgen:

In [ ]:
#Wird auf der Konsole ausgeführt
%pip install json openai requests

In [ ]:
import os.path
import requests

# Download einer Beispiel-Speisekarte, welche bereits in strukturiertem Python-Code vorliegt.
# Könnte man später durch einen Datei- oder Webseitenimport ersetzen.

MENU_DEMO_URL = "https://raw.githubusercontent.com/TAS-AG/TAS.OpenAI.Workshop/refs/heads/main/data/speisekarte_pizza_sample.py"

response = requests.get(MENU_DEMO_URL)
with open(os.path.basename(MENU_DEMO_URL), 'w', encoding='utf-8') as f:
    f.write(response.text)

from speisekarte_pizza_sample import MENU

In [ ]:
from openai import OpenAI

# Damit wir die OpenAI API nutzen können, benötigen wir einen API-Key.
# Diesen kann man sich selbst in seinem OpenAI-Konto generieren.

from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')    # Alternative 1: Google Colab
#openai_api_key = os.getenv('OPENAI_API_KEY')      # Alternative 2: Lokaler Systemumgebungsvariable, ggf. mit .env-Datei
#openai_api_key = "your_openai_api_key_here"       # Alte
# rnative 3: Direkt im Code (nicht empfohlen)

openai_client = OpenAI(api_key = openai_api_key)

# OpenAI-API verwenden

In [ ]:
import json

user_input = "Ich möchte gerne etwas mit scharfer Salami oder Pilzen."

# Hier führen wir den Kontext (Inhalt der Speisekarte) und die Benutzeranfrage zusammen, um einen Prompt zu erstellen.
prompt = f"Speisekarte:\n{json.dumps(MENU, ensure_ascii=False)}\n\nBestellung: {user_input}"

### Unstrukturierte Ausgabe

**Aufgabe**:
Führe folgenden Code mehrfach aus und beobachte die Ausgabe. Was fällt dir auf?

In [ ]:
# Unstrukturierte Ausgabe

response = openai_client.responses.create(
    model = "gpt-4o",
    input = [
        {
            "role": "developer",
            "content": "Du bist ein digitaler Kellner. Wähle alle passenden Einträge aus der Speisekarte aus."
        },
        {
            "role": "user",
            "content":  prompt
		}
    ],
)

print(response.output_text)

### Strukturierte Ausgabe

In [ ]:
# Strukturierte Ausgabe

expected_response_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",  #Erwarteter Rückgabetyp ist ein Array von Objekten
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "name": {"type": "string"},
                    "description": {"type": "string"},
                    "category": {"type": "string"},
                    "price": {"type": "number"},
                },
                "required": ["id", "name", "description", "category", "price"],
                "additionalProperties": False,
            },
        }
    },
    "required": ["items"],
    "additionalProperties": False,
}

response = openai_client.responses.create(
    model = "gpt-4o",
    input = [
        {
            "role": "developer",
            "content": "Du bist ein digitaler Kellner. Wähle den passenden Eintrag aus der Speisekarte aus."
        },
        {
            "role": "user",
            "content":  prompt
		}
    ],
    # Gewünschtes Ausgabeformat wird API-Aufruf übergeben
    text={
        "format": {
            "type": "json_schema",
            "name": "menu_item",
            "schema": expected_response_schema,
            "strict": True
        }
    }
)
recommendations = json.loads(response.output_text)["items"]

for item in recommendations:
    print(item)
